In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/MyDrive/Colab Notebooks/Monkeypox Data/Fold.zip"

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception, DenseNet169, InceptionResNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

In [ ]:
train_dir = '/content/Fold1/Train'
validation_dir = '/content/Fold1/Val'
test_dir = '/content/Fold1/Test'

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

In [ ]:


train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

Found 2142 files belonging to 2 classes.


In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Found 420 files belonging to 2 classes.


In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Found 45 files belonging to 2 classes.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
# Create the base model from the pre-trained model Xception
IMG_SHAPE = IMG_SIZE + (3,)
xception_base_model = tf.keras.applications.Xception(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
xception_base_model.trainable = False

83683744/83683744 [==============================] - 4s 0us/step


In [ ]:
# Create the base model from the pre-trained model DenseNet169
IMG_SHAPE = IMG_SIZE + (3,)
densenet169_base_model = tf.keras.applications.DenseNet169(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
densenet169_base_model.trainable = False

51877672/51877672 [==============================] - 3s 0us/step


In [ ]:
# Create the base model from the pre-trained model InceptionResNetV2
IMG_SHAPE = IMG_SIZE + (3,)
inceptionres_base_model = tf.keras.applications.InceptionResNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
inceptionres_base_model.trainable = False

219055592/219055592 [==============================] - 10s 0us/step


In [ ]:
preprocess_xception = tf.keras.applications.xception.preprocess_input
preprocess_densenet169 = tf.keras.applications.densenet.preprocess_input
preprocess_inceptionres = tf.keras.applications.inception_resnet_v2.preprocess_input

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
#feature_batch = base_model(image_batch)
xception_global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
densenet_global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
inceptionres_global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#feature_batch_average = global_average_layer(feature_batch)
#print(feature_batch_average.shape)

In [ ]:
xception_prediction_layer = Dense(1)
densenet_prediction_layer = Dense(1)
inceptionres_prediction_layer = Dense(1)
#prediction_batch = prediction_layer(feature_batch_average)
#print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))

In [ ]:
xception_output = data_augmentation(inputs)
xception_output = preprocess_xception(xception_output)
xception_output = xception_base_model(xception_output, training=False)
xception_output = xception_global_average_layer(xception_output)
xception_output = Dropout(0.2)(xception_output)
xception_output = xception_prediction_layer(xception_output)
xception_model = Model(inputs, xception_output)

In [ ]:
densenet169_output = data_augmentation(inputs)
densenet169_output = preprocess_densenet169(densenet169_output)
densenet169_output = densenet169_base_model(densenet169_output, training=False)
densenet169_output = densenet_global_average_layer(densenet169_output)
densenet169_output = Dropout(0.2)(densenet169_output)
densenet169_output = densenet_prediction_layer(densenet169_output)
densenet169_model = Model(inputs, densenet169_output)

In [ ]:
inceptionres_output = data_augmentation(inputs)
inceptionres_output = preprocess_inceptionres(inceptionres_output)
inceptionres_output = inceptionres_base_model(inceptionres_output, training=False)
inceptionres_output = inceptionres_global_average_layer(inceptionres_output)
inceptionres_output = Dropout(0.2)(inceptionres_output)
inceptionres_output = inceptionres_prediction_layer(inceptionres_output)
inceptionres_model = Model(inputs, inceptionres_output)

In [ ]:
base_learning_rate = 0.0001

#xception_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
#              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#              metrics=['accuracy'])
#densenet169_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
#              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#              metrics=['accuracy'])
#inceptionres_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
#              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#              metrics=['accuracy'])

In [ ]:
#train_data_gen = ImageDataGenerator(rescale=1./255)
#valid_data_gen = ImageDataGenerator(rescale=1./255)
#test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
#train_generator = train_data_gen.flow_from_directory(
#train_path,
#target_size=(224,224),
#batch_size=16,
#class_mode='binary')

#validation_generator = valid_data_gen.flow_from_directory(
#validation_path,
#target_size=(224,224),
#batch_size=16,
#class_mode='binary')

#test_generator = test_data_gen.flow_from_directory(
#test_path,
#target_size=(224,224),
#batch_size=16,
#class_mode='binary')

In [ ]:
checkpoint = ModelCheckpoint(
'ensemble_model_v1.h5',
monitor = 'val_accuracy',
verbose=1,
save_best_only=True,
mode='max')

In [ ]:
merge = Concatenate([xception_model, densenet169_model, inceptionres_model])

In [ ]:
predictions = Dense(1, activation='sigmoid')(merge)

TypeError: ignored

In [ ]:
ensemble_model = Model(inputs = inputs, outputs = predictions)
ensemble_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

NameError: ignored

In [ ]:
ensemble_input = Input(shape=(224,224,3))
xception_output = xception_model(ensemble_input)
densenet169_output = densenet169_model(ensemble_input)
incepresnetv2_output = inceptionres_model(ensemble_input)
ensemble_output = concatenate([xception_output, densenet169_output, inceptionres_output])
ensemble_output = Dense(128, activation='relu')(ensemble_output)
#ensemble_output = Dropout(0.5)(ensemble_output)
ensemble_output = Dense(1, activation='sigmoid')(ensemble_output)
ensemble_model = Model(ensemble_input, ensemble_output)

In [ ]:
ensemble_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
plot_model(ensemble_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
history = ensemble_model.fit(
    train_dataset,
    steps_per_epoch=len(train_dataset),
    epochs=10,
    validation_data=validation_dataset,
    validation_steps=len(validation_dataset))

In [ ]:
ensemble_model.evaluate(test_dataset, steps=len(test_dataset))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()